## **Text2Speech T2S API**

This tutorial uses ondewo-t2s-api to:

*   List the possible pipelines that can be used for synthesizing
*   List the possible languages that can be used in the synthesize process
*   List the possible domains
*   Synthesize a text to audio
*   Synthesize a batch of texts to audios
*   Manipulate pipelines (Create, Delete, Update, Get)



In [ ]:
import os
import io
import soundfile as sf
import IPython.display as ipd
import grpc
from ondewo_grpc.ondewo.t2s import text_to_speech_pb2, text_to_speech_pb2_grpc
import google.protobuf.empty_pb2 as empty_pb2
from google.protobuf.json_format import ParseDict, MessageToDict, MessageToJson

The example below shows how to create a secure channel for a text to speech stub object. When setting use_secure_channel=True, a grpc certificate grpc_cert is required.

In [ ]:
MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = "localhost" #"<ADD GRPC SERVER HERE>"
GRPC_PORT: str = "50555" #"<ADD GRPC PORT HERE>"
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"
grpc_cert: str = None #"<ADD CERTIFICATE HERE>"
credentials = grpc.ssl_channel_credentials(root_certificates=grpc_cert)

options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]


channel = grpc.insecure_channel(CHANNEL, options=options)

stub = text_to_speech_pb2_grpc.Text2SpeechStub(channel=channel)


## List all existing text to speech pipelines

All relevant configurations of the text to speech server are defined in a text to speech pipeline. A running server can store several of such configurations at the same time, and the client can chose which one to pick when he/she sends a request to synthesize a text or batch of texts.

The example below shows how to list all available pipelines by calling the ListT2sPipelines function, which takes a ListT2sPipelinesRequest as an argument and retrieves a ListT2sPipelinesResponse.

In [ ]:
pipelines = stub.ListT2sPipelines(request=empty_pb2.Empty()).pipelines

## List all possible synthesizying languages

A running server can list all possible languages fulfilling specified requirements that can be used to synthesize.

The example below shows how to list all available languages by calling the ListT2sLanguages function, which takes a ListT2sLanguagesRequest as an argument and retrieves a ListT2sLanguagesResponse.


In [ ]:
request = text_to_speech_pb2.ListT2sLanguagesRequest(speaker_sexes=['female'])
response = stub.ListT2sLanguages(request=request)

## List all possible domains

A running server can list all possible domains fulfilling specified requirements that can be used to synthesize.

The example below shows how to list all available domains by calling the ListT2sDomains function, which takes a ListT2sDomainsRequest as an argument and retrieves a ListT2sDomainsResponse.


In [ ]:
request = text_to_speech_pb2.ListT2sDomainsRequest(languages=['en'])
response = stub.ListT2sDomains(request=request)

# Make a synthesize request to the server

The running server offers a feature for synthesizying a text into a audio. In order to make use of it, the Synthesize method is utilized. This method will receive a SynthesizeRequest and retrieve a SynthesizeResponse.



The following pipelines were chosen to examplify

In [ ]:
english_pipeline = text_to_speech_pb2.T2sPipelineId(id='pipeline_id')

The example below shows how to synthesize a text into an audio.
1.   A configuration has to be created with a RequestConfig, specifying the desired optional parameters.
2.   A request has to be created with a SynthesizeRequest, specifying the text to be synthesize and the previously created configuration.
3.   By calling the Synthesize method with the created request, the text is synthesized with the specfified configuration.



In [ ]:
config = text_to_speech_pb2.RequestConfig(t2s_pipeline_id='pipeline_id', length_scale = 1.0, pcm=2, audio_format= 3)
request = text_to_speech_pb2.SynthesizeRequest(text='Hallo, wie geht es dir?', config=config)
response = stub.Synthesize(request=request)

# Make a synthesize request to the server for a batch of texts

The running server offers a feature for synthesizying a batch of texts into audios. In order to make use of it, the BatchSynthesize method is utilized. This method will receive a BatchSynthesizeRequest and retrieve a BatchSynthesizeResponse.

The example below shows how to synthesize a batch of texts into a audios.

1.   A configuration has to be created with a RequestConfig, specifying the desired optional parameters for each text in the batch.
2.   A request has to be created with a SynthesizeRequest, specifying the text to be synthesize and the previously created configuration for each text in the batch with its desired configuration.
3.  By calling the BatchSynthesize with the created request, the text is synthesized with the specfified configuration.

In [ ]:
config_1 = text_to_speech_pb2.RequestConfig(t2s_pipeline_id='pipeline_id', length_scale = 1.0, pcm=0, audio_format= 0)
config_2 = text_to_speech_pb2.RequestConfig(t2s_pipeline_id='pipeline_id', length_scale = 0.5, pcm=0, audio_format= 1)
config_3 = text_to_speech_pb2.RequestConfig(t2s_pipeline_id='pipeline_id', length_scale = 1.0, pcm=1, audio_format= 0)

request_1 = text_to_speech_pb2.SynthesizeRequest(text='Hello', config=config_1)
request_2 = text_to_speech_pb2.SynthesizeRequest(text='How are you?', config=config_2)
request_3 = text_to_speech_pb2.SynthesizeRequest(text='Hallo, wie geht es dir?', config=config_3)

request = text_to_speech_pb2.BatchSynthesizeRequest(batch_request = [request_1, request_2, request_3])

response = stub.BatchSynthesize(request = request)

## Create Pipeline

The server provides a method for creating new pipelines. This can be done with the function CreateT2sPipeline, which receives a Text2SpeechConfig and retrieves a T2sPipelineId.

In [ ]:
request = text_to_speech_pb2.Text2SpeechConfig(id='new_pipeline', description='description', active=True, inference='inference', normalization='normalization', postprocessing='postprocessing')
new_pipeline_id = stub.CreateT2sPipeline(request=request)

## Get Pipeline

In order to get an specific pipeline configuration the GetT2sPipeline method is used. This method received a T2sPipelineId and retrieves a Text2SpeechConfig.

In [ ]:
request = text_to_speech_pb2.T2sPipelineId(id='pipeline_id')
pipeline_config = stub.GetT2sPipeline(request=request)

## Update Pipeline

The server provides a method to update a pipeline called UpdateT2sPipeline, receiving a pipeline configuration.

In the following example, the retrieved configuration in the previous call is modified and used to update the pipeline.

pipeline_config.inference.composite_inference.text2mel.glow_tts.length_scale = 2

In [ ]:
stub.UpdateT2sPipeline(request=pipeline_config)

## Delete Pipeline

A pipeline can be deleted with the method DeleteT2sPipeline, receiving a pipeline id.

In [ ]:
stub.DeleteT2sPipeline(request=request)